In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, KFold
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from skopt.space import Real, Integer, Categorical
from skopt import gp_minimize

import utils.dev_config as dev_conf
import utils.preprocessing as prep
import utils.optimization as opt

In [2]:
dirs = dev_conf.get_dev_directories("../dev_paths.txt")
unified_dsets = ["unified_cervical_data", "unified_uterine_data", "unified_uterine_endometrial_data"]
matrisome_list = f"{dirs.data_dir}/matrisome/matrisome_hs_masterlist.tsv"

In [3]:
i = 0

In [4]:
matrisome_df = prep.load_matrisome_df(matrisome_list)

In [5]:
seed = 123
rand = np.random.RandomState()

# Load and filter survival data

In [6]:
event_code = {"Alive": 0, "Dead": 1}
covariate_cols = ["figo_stage", "age_at_diagnosis", "race", "ethnicity"]
dep_cols = ["vital_status", "survival_time"]
cat_cols = ["race", "ethnicity", "figo_chr"]
survival_df = prep.load_survival_df(f"{dirs.data_dir}/{unified_dsets[i]}/survival_data.tsv", event_code)

In [7]:
filtered_survival_df = (
    prep.decode_figo_stage(survival_df[["sample_name"] + dep_cols + covariate_cols].dropna(), to="c")
        .query("vital_status == 1")
        .drop(["vital_status"], axis=1)
        .pipe(pd.get_dummies, columns=cat_cols)
        .reset_index(drop = True)
)
filtered_survival_df.columns = filtered_survival_df.columns.str.replace(' ', '_')

print(filtered_survival_df.shape)
# filtered_survival_df.head()

(66, 16)


# Load normalized matrisome count data

In [8]:
norm_matrisome_counts_df = pd.read_csv(f"{dirs.data_dir}/{unified_dsets[i]}/norm_matrisome_counts.tsv", sep='\t')
norm_filtered_matrisome_counts_t_df = prep.transpose_df(
    norm_matrisome_counts_df[["geneID"] + list(filtered_survival_df.sample_name)], "geneID", "sample_name"
)
print(norm_filtered_matrisome_counts_t_df.shape)
# norm_filtered_matrisome_counts_t_df.head()

(66, 1009)


# Join survival and count data

In [9]:
joined_df = (
    pd.merge(filtered_survival_df, norm_filtered_matrisome_counts_t_df, on="sample_name")
        .set_index("sample_name")
)
print(joined_df.shape)
# joined_df.head()

(66, 1023)


# Optimize model

In [10]:
rand.seed(seed)
x_df, y_df = prep.shuffle_data(joined_df, rand)

In [11]:
def objective(h_params, X, y, r, verbose=True):
    if verbose:
        print(h_params)
    model = RandomForestRegressor(
        n_estimators=h_params[0],
        max_depth=h_params[1],
        max_features=h_params[2],
        min_samples_split=h_params[3],
        min_samples_leaf=h_params[4],
        bootstrap=h_params[5],
        n_jobs=-1,
        random_state=r
    )
    return -np.mean(cross_val_score(
        model,
        X,
        y,
        cv=KFold(n_splits=5),
        n_jobs=-1,
        scoring="neg_mean_absolute_error"
    ))

In [12]:
space = [
    Integer(int(1e2), int(1e3), name="n_estimators"),
    Integer(int(10), int(100), name="max_depth"),
    Categorical(["auto", "sqrt", "log2"], name="max_features"),
    Integer(int(2), int(4), name="min_samples_split"),
    Integer(int(1), int(3), name="min_samples_leaf"),
    Categorical([True, False], name="bootstrap")
]
n_initial = 10 * len(space)
n_calls = 50 * len(space)

In [13]:
callback_file = f"{unified_dsets[i]}_opt_rfr_h_params.tsv"
try:
    os.remove(callback_file)
except OSError:
    pass


res = gp_minimize(
    lambda h_ps: objective(h_ps, x_df, y_df, rand),
    space,
    verbose=True,
    random_state=rand,
    n_initial_points=n_initial,
    n_calls = n_calls,
    n_jobs=-1,
    callback=lambda x: opt.save_callback(x, callback_file, n = 5, sep="\t")
)

Iteration No: 1 started. Evaluating function at random point.
[776, 53, 'sqrt', 3, 2, False]
Iteration No: 1 ended. Evaluation done at random point.
Time taken: 2.8527
Function value obtained: 589.0500
Current minimum: 589.0500
Iteration No: 2 started. Evaluating function at random point.
[407, 72, 'sqrt', 3, 2, False]
Iteration No: 2 ended. Evaluation done at random point.
Time taken: 1.7262
Function value obtained: 586.9204
Current minimum: 586.9204
Iteration No: 3 started. Evaluating function at random point.
[853, 34, 'log2', 2, 2, True]
Iteration No: 3 ended. Evaluation done at random point.
Time taken: 3.1785
Function value obtained: 578.9602
Current minimum: 578.9602
Iteration No: 4 started. Evaluating function at random point.
[262, 54, 'auto', 3, 1, True]
Iteration No: 4 ended. Evaluation done at random point.
Time taken: 2.3326
Function value obtained: 613.9540
Current minimum: 578.9602
Iteration No: 5 started. Evaluating function at random point.
[995, 53, 'log2', 3, 2, True

/home/carcook/anaconda3/envs/TCGA-round1/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


*****redundant*****
Iteration No: 64 ended. Search finished for the next optimal point.
Time taken: 1.9485
Function value obtained: 579.7737
Current minimum: 565.5922
Iteration No: 65 started. Searching for the next optimal point.
[1000, 15, 'sqrt', 4, 3, True]
Iteration No: 65 ended. Search finished for the next optimal point.
Time taken: 3.8771
Function value obtained: 582.5421
Current minimum: 565.5922
Iteration No: 66 started. Searching for the next optimal point.
[100, 87, 'log2', 2, 3, False]
Iteration No: 66 ended. Search finished for the next optimal point.
Time taken: 1.7550
Function value obtained: 562.1262
Current minimum: 562.1262
Iteration No: 67 started. Searching for the next optimal point.
[137, 10, 'log2', 4, 3, False]
Iteration No: 67 ended. Search finished for the next optimal point.
Time taken: 2.0401
Function value obtained: 563.9011
Current minimum: 562.1262
Iteration No: 68 started. Searching for the next optimal point.
[983, 12, 'log2', 4, 3, False]
Iteration No

/home/carcook/anaconda3/envs/TCGA-round1/lib/python3.8/site-packages/skopt/optimizer/optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Iteration No: 180 ended. Search finished for the next optimal point.
Time taken: 6.6408
Function value obtained: 569.5852
Current minimum: 551.2295
Iteration No: 181 started. Searching for the next optimal point.
[110, 100, 'sqrt', 4, 3, True]
Iteration No: 181 ended. Search finished for the next optimal point.
Time taken: 5.0691
Function value obtained: 576.3672
Current minimum: 551.2295
Iteration No: 182 started. Searching for the next optimal point.
[990, 99, 'log2', 4, 2, False]
Iteration No: 182 ended. Search finished for the next optimal point.
Time taken: 5.9173
Function value obtained: 571.3683
Current minimum: 551.2295
Iteration No: 183 started. Searching for the next optimal point.
[189, 100, 'auto', 2, 2, True]
Iteration No: 183 ended. Search finished for the next optimal point.
Time taken: 5.7019
Function value obtained: 606.6591
Current minimum: 551.2295
Iteration No: 184 started. Searching for the next optimal point.
[102, 11, 'log2', 4, 3, True]
Iteration No: 184 ended. 